In [1]:
!pip install egttools
import numpy as np
import matplotlib.pyplot as plt
from typing import Union, List

from egttools.games import AbstractNPlayerGame
from egttools import sample_simplex, calculate_nb_states
from egttools.analytical import PairwiseComparison
from egttools.utils import calculate_stationary_distribution
from egttools.plotting import draw_invasion_diagram

Defaulting to user installation because normal site-packages is not writeable


In [6]:
class PGGWithLongCommitment(AbstractNPlayerGame):
    def __init__(self, 
                 group_size: int,   # Number of participants in the PGG
                 c: float,          # Cost of cooperation
                 r: float,          # Enhancing factor (multiplier)
                 eps: float,        # Cost to propose a commitment
                 delta: float,      # Cost for not respecting the commitment
                 F: int
                 ):
        # Initialize superclass
        AbstractNPlayerGame.__init__(self, 5, group_size)  # Adjusted for additional strategies

        # Parameters and configurations
        self.nb_strategies_ = 5
        self.group_size_ = group_size
        self.strategies = [
            "COMPF","C", "D", "FAKE", "FREE"
        ]

        self.c = c
        self.r = r
        self.eps = eps
        self.delta = delta
        self.F = F 
        self.nb_group_configurations_ = self.nb_group_configurations()  # Calculate number of possible group configurations
        self.calculate_payoffs()  # Calculate payoffs for each strategy in different group configurations
    def get_strategy(self,
                        group_composition: Union[List[int], np.ndarray],
                        strategy: str,
                        ):
            
            propose_commit = False # D (but also FAKE)
            accept_commit = False # D
            contribute = False # D (but also FAKE)
            
            if "COMP" in strategy:
                propose_commit = True
                accept_commit = True
                contribute=True
            elif strategy=="C":
                accept_commit = True
                contribute=True
            elif strategy=="FREE":
                accept_commit = True
                if group_composition[0]>0:
                    contribute=True
            elif strategy=="FAKE":
                accept_commit = True

            return propose_commit, accept_commit, contribute
    

        
    def play(self, group_composition: Union[List[int], np.ndarray], game_payoffs: np.ndarray) -> None:
        # Initialize payoffs for each strategy in the group
        game_payoffs[:] = 0.
        # Calculate the number of each type of player in the group
        nb_commitment = group_composition[0]

        if nb_commitment == 0:  # Classical PGG
            nb_contributors = group_composition[1]
        else:  # With commitments
            nb_contributors = nb_commitment + group_composition[1] + group_composition[4]

        nb_fake = group_composition[3]  # Number of fake players
        nb_accept = nb_fake + nb_contributors

        # Calculate the total contribution and reward for the group
        total_contribution = self.c * nb_contributors
        total_reward = self.r * total_contribution
        individual_reward = total_reward / self.group_size_
        # Determine F (minimum commitment threshold) and F_prime
        #F = next((i + 1 for i, count in enumerate(group_composition[:25]) if count > 0), 0)
        #F_prime = next((int(self.strategies[i].split('_')[-1]) for i in range(25) if group_composition[i] > 0 and '_' in self.strategies[i]), 0)
        if nb_commitment==0 or self.F<=nb_accept:
           for index, strategy_count in enumerate(group_composition):
            if strategy_count > 0:
                propose_commit, accept_commit, contribute = self.get_strategy(group_composition, self.strategies[index]) 
                game_payoffs[index] += individual_reward
                if contribute:
                    game_payoffs[index] -=  self.c
                if propose_commit:
                     game_payoffs[index] -=  ((self.eps) - (nb_fake*self.delta))/nb_commitment
                if accept_commit and not contribute:
                    game_payoffs[index] -= self.delta
    def calculate_payoffs(self) -> np.ndarray:
        """Calculate and store the payoffs for each strategy in the game."""

        # Initialize an array to store payoffs for each configuration
        payoffs_container = np.zeros(shape=(self.nb_strategies_,), dtype=np.float64)

        # Loop over all possible group configurations
        for i in range(self.nb_group_configurations_):
            # Generate a sample group composition
            group_composition = sample_simplex(i, self.group_size_, self.nb_strategies_)
            group_composition = np.array(group_composition, dtype=float)

            # Play the game with the given group composition
            self.play(group_composition, payoffs_container)

            # Update the payoff for each strategy based on this configuration
            for strategy_index, strategy_payoff in enumerate(payoffs_container):
                self.update_payoff(strategy_index, i, strategy_payoff)

            # Reset the payoff container for the next configuration
            payoffs_container[:] = 0

        return self.payoffs()


In [30]:
group_size = 5
c = 1.2539
Z = 100  # Population size
beta = 0.25  # Selection intensity
r_c = 2.5
nb_points = 6
eps_values = np.linspace(0., 2., nb_points)
delta_values = np.linspace(0., 6., nb_points)

optimal_F_r25 = np.zeros((nb_points, nb_points), dtype = float)

for i, eps in enumerate(eps_values):
    for j, delta in enumerate(delta_values):
        optimal_F = np.zeros((5,), dtype = float)
        for F in range(5):
            game = PGGWithLongCommitment(group_size, c, r_c, eps, delta,F+1)
            evolver = PairwiseComparison(Z, game)
            transition_matrix, _ = evolver.calculate_transition_and_fixation_matrix_sml(beta)
            stationary_distribution = calculate_stationary_distribution(transition_matrix.transpose())
            optimal_F[F] += stationary_distribution[0]
        optimal_F_r25[i, j] = np.argmax(optimal_F) +1
        print(optimal_F_r25)
print(optimal_F_r25)

[[4. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[4. 4. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[4. 4. 4. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[4. 4. 4. 4. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[4. 4. 4. 4. 4. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[4. 4. 4. 4. 4. 4.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[4. 4. 4. 4. 4. 4.]
 [4. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[4. 4. 4. 4. 4. 4.]
 [4. 4. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0